# Desenhando com o `pandas`

Assume-se um conhecimento inicial da biblioteca `pandas`, veja as [notas de aula](intro_pandas.ipynb).


In [ ]:
import pandas as pd

# importa os dados removendo a coluna de índice e sort_no
movies = pd.read_csv("data/movies.csv").drop(columns=['id', 'sort_no'])
movies


## Limpando os dados


In [ ]:
movies.info()


Primeiramente vamos melhorar essas colunas...
Temos datas que estão com formato `object` (que é uma string).
Temos categorias assim também.
Temos algo que parece ser um número, mas que não está assim `user_score`.


In [ ]:
# Considerando categorias

movies['rating'].unique()


Aqui parece um bom lugar para transformar todos os _PG-13_, _PG--13_ e _PG13\`_ em uma coisa só.


In [ ]:
# Muda todos os ratings parecidos em algo padronizado
movies['rating'] = movies['rating'].map(
    lambda x: "PG-13" if x in ['PG--13', 'PG-13`'] else x)
movies['rating'].unique()


In [ ]:
# Muda a coluna rating para uma categoria, que torna as operações mais rápidas
movies['rating'] = movies['rating'].astype('category')
movies['rating'].dtype


In [ ]:
# Remove todos os filmes que não tem algum valor faltando
movies = movies.dropna()
movies


In [ ]:
# user_score tem notas que são tbd em vez de números
movies['user_score'].unique()


In [ ]:
list_to_drop = movies['user_score'] == "tbd"
movies[list_to_drop]
movies = movies.drop(movies[list_to_drop].index, axis=0)
movies


In [ ]:
# Agora sem os valores não numéricos podemos transformar essa coluna em numérica
movies['user_score'] = pd.to_numeric(movies['user_score'])
movies.info()


Sumário e título não precisamos mexer...
Fica faltando a data.


In [ ]:
movies['release_date'] = pd.to_datetime(movies['release_date'])
movies['release_date']


In [ ]:
movies.info()


In [ ]:
# Só falta reiniciar o índice pois como deletamos índices melhor ter os índices sem pulos
movies = movies.reset_index(drop=True)
movies


## Desenhos

É possível utilizar a biblioteca `matplotlib` para fazer os desenhos, mas o próprio pandas também é capaz de fazer alguns deles usando os métodos `plot`.


In [ ]:
# Agrupamos todos os valores por classificação indicadora
group_by_rating = movies.groupby('rating')
group_by_rating.describe()


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# Agora podemos desenhar um gráfico de barras representando a quantidade de filmes baseado na classificação indicadora
movies['rating'].value_counts() \
    .head(10) \
    .plot \
    .barh(xlabel="Quantidade", ylabel="Classificação", title="Quantidade de Filmes por Classificação")
plt.show()


In [ ]:
# Ou podemos também ver o score médio baseado na classificação
axes = movies['release_date'].map(lambda x: x.year) \
    .plot \
    .hist(bins=40, title="Quantidade de Filmes por Ano")
axes.set_ylabel("Quantidade")
axes.set_xlabel("Ano")
plt.show()


In [ ]:
# Agora relacionamos o user score com o metascore

axes = movies.plot.scatter(x="user_score", y="metascore",
                           title="User Score vs MetaScore", xlabel="User Score", ylabel="MetaScore")
plt.show()


In [ ]:
# Podemos comparar o score médio do usuário e dos críticos com a classificação geral do filme

grouped = movies.groupby('rating', sort=True)[
    ['metascore', 'user_score']].mean()
grouped['metascore'] = grouped['metascore'].map(lambda x: x / 10)
axes = grouped.plot.barh(
    xlabel="Nota", ylabel="Classificação", title="Notas por Classificação")
axes.legend(labels=["críticos", "usuários"])
plt.show()


In [ ]:
# Finalmente uma tabela de correlação entre os scores
movies[['metascore', 'user_score']].corr()
